In [1]:
import os
import sys
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.abspath(os.path.join('', 'summary')))
sys.path.insert(0, os.path.abspath(os.path.join('', 'utils')))
sys.path.insert(0, os.path.abspath(os.path.join('', '..', 'cdm-utilities')))
sys.path.insert(0, os.path.abspath(os.path.join('', '..', 'cdm-utilities', 'minio_api')))
from data_classes_cdm import CDMProcessingVariables as config_cdm
from minio_api import MinioAPI
from utils import mrn_zero_pad, print_df_without_index, set_debug_console
from get_anchor_dates import get_anchor_dates
from create_summary_from_redcap_reports import RedcapToCbioportalFormat
from cbioportal_summary_file_combiner import cbioportalSummaryFileCombiner



In [2]:
set_debug_console()



In [7]:
# ## Define hardcoded variables
# ### Filenames
# #### Redcap report manifest files
# FNAME_MANIFEST_PATIENT = 'cbioportal/summary_manifest_patient_test.csv'
# FNAME_MANIFEST_SAMPLE = 'cbioportal/summary_manifest_sample_test.csv'
# #### Current summary files
# FNAME_SUMMARY_TEMPLATE_P = config_cdm.fname_p_sum_template
# FNAME_SUMMARY_TEMPLATE_S = config_cdm.fname_s_sum_template
# #### Summary files to save (datahub)
# FNAME_SUMMARY_P = config_cdm.fname_summary_patient + '.tsv'
# FNAME_SUMMARY_S = config_cdm.fname_summary_sample + '.tsv'
# #### Summary files to save (MinIO)
# FNAME_SUMMARY_P_MINIO = config_cdm.fname_summary_patient_minio
# FNAME_SUMMARY_S_MINIO = config_cdm.fname_summary_sample_minio
# MINIO
ENV_MINIO = config_cdm.minio_env
# Data dictionary
path = '/mind_data/cdm_repos/cdm-utilities/docs/dev/'
fname_metadata = os.path.join(path, 'CDM-Codebook - metadata.csv')
fname_project = os.path.join(path, 'CDM-Codebook - project.csv')
fname_tables = os.path.join(path, 'CDM-Codebook - tables.csv')

In [8]:
# ### Columns of interest for joining or removing
# COL_PID = 'DMP_ID'
# COL_PID_CBIO = 'PATIENT_ID'




In [9]:
patient_or_sample = 'patient'




In [10]:
obj_minio = MinioAPI(fname_minio_env=ENV_MINIO)


q8d0
c8BW


In [5]:
obj_format_cbio = RedcapToCbioportalFormat()




NameError: name 'RedcapToCbioportalFormat' is not defined

In [6]:
df_manifest = obj_format_cbio.return_manifest()
df_manifest


NameError: name 'obj_format_cbio' is not defined

In [7]:
obj_format_cbio.create_summaries_and_headers(
    patient_or_sample=patient_or_sample
)

NameError: name 'obj_format_cbio' is not defined

In [8]:
df_manifest_s = obj_format_cbio.return_manifest()

NameError: name 'obj_format_cbio' is not defined

In [9]:
df_manifest_s

NameError: name 'df_manifest_s' is not defined

In [10]:
df_metadata = pd.read_csv(fname_metadata)
df_project = pd.read_csv(fname_project)
df_tables = pd.read_csv(fname_tables)

NameError: name 'fname_metadata' is not defined

In [9]:
df_anchor = get_anchor_dates()



Loading pathology/table_pathology_clean.tsv


In [13]:
# Get form names that contain data elements to host on cbioportal
logic_for_cbio = df_metadata['for_cbioportal'] == 'x'
forms = df_metadata.loc[logic_for_cbio, 'form_name'].unique()

# From form names, get corresponding tables and their filenames
f1 = df_tables['form_name'].isin(forms)


if patient_or_sample == 'sample':
    id_label = '#Sample Identifier'
    fname_manifest = FNAME_MANIFEST_SAMPLE
    fname_template = FNAME_SUMMARY_TEMPLATE_S
    f2 = df_tables['cbio_summary_id_sample'].notnull()
    col_id_change = 'SAMPLE_ID'
elif patient_or_sample == 'patient':
    id_label = '#Patient Identifier'
    fname_manifest = FNAME_MANIFEST_PATIENT
    fname_template = FNAME_SUMMARY_TEMPLATE_P
    f2 = df_tables['cbio_summary_id_sample'].isnull()
    col_id_change = 'PATIENT_ID'
    
    
active_tables = df_tables.loc[f1&f2]
list_fname_minio = active_tables['cdm_source_table']
list_fname_minio

1                     demographics/ddp_demographics.tsv
8           diagnosis/IMPACT_Darwin_Patient_Summary.tsv
9                  diagnosis/table_met_site_summary.tsv
16                  insurance/ddp_insurance_summary.tsv
31             medications/table_tx_summary_patient.tsv
35    comorbidities/ddp_comorbidities_index_summary.tsv
44         comorbidities/smoking_status_predictions.tsv
Name: cdm_source_table, dtype: object

In [14]:
# Initialize the manifest file
obj_format_cbio.summary_manifest_init()

for i,fname in enumerate(list_fname_minio):
    obj = obj_minio.load_obj(path_object=fname)
    df_ = pd.read_csv(obj, header=0, low_memory=False, sep='\t')
    form = df_tables.loc[list_fname_minio.index[i], 'form_name']
    # print('-----------------------')
    print(fname)
    # print(form)
    filt_table_use = logic_for_cbio & (df_metadata['form_name'] == form)
    filt_col_dates = df_metadata.loc[filt_table_use]['text_validation_type_or_sh'] == 'date_mdy'
    cols_to_use = list(df_metadata.loc[filt_table_use, 'field_name'])
    cols_dates = list(df_metadata.loc[filt_table_use&filt_col_dates, 'field_name'])
    key_patient = active_tables['cbio_summary_id_patient'].iloc[i]
    key_sample = active_tables['cbio_summary_id_sample'].iloc[i]
    
    if pd.isna(key_sample):
        
        key = [key_patient]
    else:
        key = [key_sample, key_patient]
        
    cols_filter = key + list(cols_to_use)    
    
    # print(key)
    # print(cols_filter)
    # print(cols_dates)
    
    # Merge with anchor dates and de-id
    df_select = df_[cols_filter]
    
    if (key_patient == 'MRN'):
        df_select = mrn_zero_pad(df=df_select, col_mrn='MRN')
        df_select = df_select.merge(right=df_anchor, how='inner', on='MRN')
        df_select = df_select.drop(columns=['MRN'])
    else:
        df_anchor1 = df_anchor.drop(columns=['MRN'])
        df_select = df_select.merge(
            right=df_anchor1, 
            how='inner', 
            left_on=key_patient, 
            right_on='DMP_ID'
        ) 
    # Convert dates to intervals
    df_select[cols_dates] = df_select[cols_dates].apply(lambda x: pd.to_datetime(x, errors='coerce'))
    df_select[cols_dates] = df_select[cols_dates].apply(lambda x: (x - df_select['DTE_PATH_PROCEDURE']).dt.days)
    df_select = df_select.drop(columns=['DTE_PATH_PROCEDURE'])
    df_select = df_select.rename(columns={'DMP_ID': COL_PID_CBIO})
    df_select.columns = [x.upper().replace(' ', '_') for x in df_select.columns]
    
    
    # Create header
    cols_header = list(df_select.columns)
    df_header = obj_format_cbio._format_data_dictionary(
        df=df_metadata[filt_table_use]
    )
    length_header = df_header.shape[0]
    df_header.loc[length_header, 'label'] = id_label
    df_header.loc[length_header, 'data_type'] = np.NaN
    df_header.loc[length_header, 'comment'] = col_id_change
    df_header.loc[length_header, 'heading'] = col_id_change
    # Reorder header entries to match summary file
    sorterIndex = dict(zip(cols_header, range(len(cols_header))))
    df_header['Rank'] = df_header['heading'].map(sorterIndex)
    df_header.sort_values(by='Rank', ascending = True, inplace = True)
    df_header.drop(columns=['Rank', 'is_date'], axis=1, inplace = True)
    df_header['heading'] = df_header['heading'].str.upper().str.replace(' ', '_')
    
    # Reorder data columns to the header
    df_select = df_select[list(df_header['heading'])]
    

    # Create manifest file
    ## Modify/create manifest files --------------------------------------------------
    fname_save_data = 'cbioportal/' + form.lower().replace(' ','_') + '_data.csv'
    fname_save_header = 'cbioportal/' + form.lower().replace(' ','_') + '_header.csv'
    ##### This file will be used for merging data    
    obj_format_cbio.summary_manifest_append(
        instr_name=form,
        fname_df_save=fname_save_data,
        fname_header_save=fname_save_header
    )

    ## Save data and header files --------------------------------------------------
    ### Save cbioportal formatted patient level dx data and header files 
    print('Saving %s' % fname_save_data)
    print(df_select.head())
    obj_minio.save_obj(df=df_select, 
                       path_object=fname_save_data, 
                       sep=',')
    print('Saving %s' % fname_save_header)
    print(df_header.head())
    obj_minio.save_obj(df=df_header, 
                       path_object=fname_save_header, 
                       sep=',')
    
    
## Note: Continue appending manifest files here
df_manifest_s = obj_format_cbio.return_manifest()
##### Save manifest
obj_format_cbio.summary_manifest_save(fname_save=fname_manifest)    
    
    

demographics/ddp_demographics.tsv


/mind_data/cdm_repos/cdm-utilities/utils.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_mrn] = df[col_mrn].astype(str).str.zfill(8)


Saving cbioportal/demographics_data.csv
   PT_DEATH_DTE  MRN_CREATE_DTE  GENDER MARITAL_STATUS        RELIGION   RACE                                          ETHNICITY  PLA_LAST_CONTACT_DTE  PLA_LAST_ACTV_DTE  PLA_LAST_ADM_DTE  PLA_LAST_DSCH_DTE  PLA_LAST_APPT_DTE  PLA_LAST_DRVST_DTE  PLA_LAST_TX_DTE  PLA_LAST_SURG_DTE  PLA_LAST_RT_DTE PATIENT_ID
0           NaN         -9054.0    MALE         SINGLE  CATHOLIC/ROMAN  WHITE  Non-Spanish; Non-Hispanic                     ...                 172.0              326.0           -8845.0            -8844.0              172.0               158.0            326.0              326.0          -8813.0  P-0078271
1           NaN         -6916.0    MALE         SINGLE            NONE  WHITE  Non-Spanish; Non-Hispanic                     ...                2123.0             2123.0               0.0                3.0             2123.0              2123.0              0.0                0.0              NaN  P-0015526
2         333.0         -6090.

/mind_data/cdm_repos/cdm-utilities/utils.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_mrn] = df[col_mrn].astype(str).str.zfill(8)


Saving cbioportal/smoking_history_data.csv
  PATIENT_ID SMOKING_PREDICTIONS_3_CLASSES SMOKING_PREDICTIONS_4_CLASSES
0  P-0086671                         Never                         Never
1  P-0079052                         Never                         Never
2  P-0079026                         Never                         Never
3  P-0078287                         Never                         Never
4  P-0082629                         Never                         Never
Saving cbioportal/smoking_history_header.csv
                           label                                            comment data_type  visible                        heading
2            #Patient Identifier                                         PATIENT_ID       NaN      NaN                     PATIENT_ID
465  Smoking History (3 Classes)  Smoking history classes: Current/Former, Never...    STRING      1.0  SMOKING_PREDICTIONS_3_CLASSES
466  Smoking History (4 Classes)  Smoking history classes: Current, Form

In [11]:
df_manifest_s

,REPORT_NAME,SUMMARY_FILENAME,SUMMARY_HEADER_FILENAME
0,Demographics,cbioportal/demographics_data.csv,cbioportal/demographics_header.csv
1,Patient Diagnosis Summary,cbioportal/patient_diagnosis_summary_data.csv,cbioportal/patient_diagnosis_summary_header.csv
2,Metastatic Site Summary,cbioportal/metastatic_site_summary_data.csv,cbioportal/metastatic_site_summary_header.csv
3,Insurance Summary,cbioportal/insurance_summary_data.csv,cbioportal/insurance_summary_header.csv
4,Medication Patient Summary,cbioportal/medication_patient_summary_data.csv,cbioportal/medication_patient_summary_header.csv
5,Comorbidity Index Score,cbioportal/comorbidity_index_score_data.csv,cbioportal/comorbidity_index_score_header.csv
6,Smoking History,cbioportal/smoking_history_data.csv,cbioportal/smoking_history_header.csv


In [15]:
## Sample summary merging
# obj_s_combiner = cbioportalSummaryFileCombiner(
#     fname_minio_env=ENV_MINIO,
#     fname_manifest=fname_manifest, 
#     fname_current_summary=fname_template, 
#     patient_or_sample=patient_or_sample
# )

# obj_s_combiner.save_update(fname=FNAME_SUMMARY_S)
# df_cbio_summary_s = obj_s_combiner.return_final()